In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import time

df = pd.read_csv('dataset.csv')

In [58]:
df = df.drop(columns=['Date'])
target = 'Revenue_(USD)'
X = df.drop(columns=[target])
y = df[target]
display(X)

,Average_views_per_viewer,Unique_viewers,Click_rate_(%),Impressions,Comments,Shares,Dislikes,Likes,Average_viewed_(%),Videos_published,Views,Watch_hours,Average_view_sec,Subs_accumulated
0,1.1538,13.0,7.38,122.0,2.0,1.0,0.0,5.0,53.59,1.0,15.0,0.4666,111.0,5.0
1,1.1935,31.0,23.14,121.0,3.0,0.0,0.0,4.0,44.58,0.0,37.0,0.9575,93.0,9.0
2,1.3684,19.0,20.65,92.0,1.0,0.0,0.0,5.0,38.75,0.0,26.0,0.5849,80.0,11.0
3,1.0000,8.0,13.64,44.0,0.0,0.0,0.0,0.0,24.81,0.0,8.0,0.1152,51.0,11.0
4,1.0625,16.0,13.89,72.0,0.0,0.0,0.0,0.0,45.50,0.0,17.0,0.4491,95.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2.2721,105872.0,4.04,4376635.0,546.0,399.0,195.0,10594.0,25.75,0.0,240554.0,19512.3516,292.0,565123.0
1128,1.8941,215712.0,5.83,4854638.0,2741.0,764.0,312.0,34422.0,30.35,4.0,408582.0,19855.7704,174.0,566855.0
1129,2.1319,305881.0,6.08,7768271.0,4741.0,1055.0,521.0,56677.0,47.81,0.0,652105.0,54463.2654,300.0,570139.0
1130,2.3468,152920.0,4.51,5788014.0,671.0,633.0,276.0,18458.0,41.24,0.0,358868.0,26757.4077,268.0,572582.0


#### Chuẩn bị các bộ dữ liệu

In [59]:
# a. Dữ liệu gốc (Đã chuẩn hóa)
# Chuẩn hóa là bắt buộc đối với K-NN và PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Số chiều dữ liệu gốc: {X_scaled.shape[1]}")

Số chiều dữ liệu gốc: 14


In [60]:
# b. Dữ liệu giảm chiều (PCA)
# Giữ lại 95% phương sai
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
print(f"Số chiều sau khi giảm bằng PCA (giữ 95% phương sai): {X_pca.shape[1]}")

Số chiều sau khi giảm bằng PCA (giữ 95% phương sai): 6


#### Thiết lập Thử nghiệm

In [61]:
# Định nghĩa các bộ dữ liệu sẽ chạy
datasets = {
    "Dữ liệu gốc (Đã chuẩn hóa)": X_scaled,
    "Dữ liệu giảm chiều (PCA)": X_pca
}

# Định nghĩa các tỷ lệ chia
splits = {
    "4:1 (test=20%)": 0.2,
    "7:3 (test=30%)": 0.3,
    "6:4 (test=40%)": 0.4
}

# Danh sách để lưu kết quả
results = []

#### Vòng lặp Huấn luyện và Đánh giá

In [62]:
print("\nBắt đầu huấn luyện và đánh giá các mô hình...")
start_time = time.time()

# Vòng lặp qua các bộ dữ liệu (gốc và PCA)
for data_name, X_data in datasets.items():
    print(f"\nĐang xử lý: {data_name}")

    # Vòng lặp qua các tỷ lệ chia
    for split_name, test_size in splits.items():

        # Chia dữ liệu train/test
        X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=test_size, random_state=42)

        # Khởi tạo mô hình K-NN Regressor
        knn = KNeighborsRegressor()

        # Huấn luyện mô hình
        knn.fit(X_train, y_train)

        # Đánh giá mô hình tốt nhất trên tập train và test
        y_train_pred = knn.predict(X_train)
        y_test_pred = knn.predict(X_test)

        # Tính toán các chỉ số
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

        # Lưu kết quả
        results.append({
            "Loại dữ liệu": data_name,
            "Tỷ lệ (Train:Val)": split_name,
            "Train R2": train_r2,
            "Test R2": test_r2,
            "Train RMSE (USD)": train_rmse,
            "Test RMSE (USD)": test_rmse
        })


end_time = time.time()
print(f"\nHoàn thành toàn bộ quá trình sau {end_time - start_time:.2f} giây.")
display(results)


Bắt đầu huấn luyện và đánh giá các mô hình...

Đang xử lý: Dữ liệu gốc (Đã chuẩn hóa)

Đang xử lý: Dữ liệu giảm chiều (PCA)

Hoàn thành toàn bộ quá trình sau 0.05 giây.


[{'Loại dữ liệu': 'Dữ liệu gốc (Đã chuẩn hóa)',
  'Tỷ lệ (Train:Val)': '4:1 (test=20%)',
  'Train R2': 0.9793109707925217,
  'Test R2': 0.9816582429893114,
  'Train RMSE (USD)': np.float64(6.839698269041983),
  'Test RMSE (USD)': np.float64(5.8020232710720565)},
 {'Loại dữ liệu': 'Dữ liệu gốc (Đã chuẩn hóa)',
  'Tỷ lệ (Train:Val)': '7:3 (test=30%)',
  'Train R2': 0.9752613001775767,
  'Test R2': 0.9796216230467855,
  'Train RMSE (USD)': np.float64(7.408381811236555),
  'Test RMSE (USD)': np.float64(6.514698309758979)},
 {'Loại dữ liệu': 'Dữ liệu gốc (Đã chuẩn hóa)',
  'Tỷ lệ (Train:Val)': '6:4 (test=40%)',
  'Train R2': 0.975465886967962,
  'Test R2': 0.9787228141583618,
  'Train RMSE (USD)': np.float64(7.14039570553997),
  'Test RMSE (USD)': np.float64(7.0422139085374456)},
 {'Loại dữ liệu': 'Dữ liệu giảm chiều (PCA)',
  'Tỷ lệ (Train:Val)': '4:1 (test=20%)',
  'Train R2': 0.9782015586570464,
  'Test R2': 0.9784058882624763,
  'Train RMSE (USD)': np.float64(7.020686937906317),
  'Test